<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


In [ ]:
import io
#df = pd.read_csv('Apple60.csv', skiprows=0)

from sklearn.model_selection import train_test_split
data = pd.read_csv('Apple60.csv', sep=',', header=0)
test_data = pd.read_csv('Apple60_Major.csv', sep=',', header=0)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
data[:10]

,Source,News,Movement
0,www.businesswire.com,NAPE Summit Week will also feature the annual ...,1
1,seekingalpha.com,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,0
2,seekingalpha.com,The areas to be concerned are that there are f...,1
3,seekingalpha.com,Amazon will soon resume selling the Apple TV (...,1
4,seekingalpha.com,"With such a large revenue pie, naturally we be...",0
5,seekingalpha.com,"Facing questions from tech analysts, Apple (NA...",0
6,www.thestreet.com,Cramer and the AAP team say investors need to ...,0
7,seekingalpha.com,One day after Apple acknowledged that it has b...,0
8,www.thestreet.com,1 . -- U.S. stock futures turned lower on Tues...,0
9,seekingalpha.com,"Meanwhile, eight more lawsuits have been filed...",0


In [ ]:
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

MAX_NB_WORDS = 40000
max_seq_len = 1000

raw_docs_train = train_df['News'].tolist()
raw_docs_test = test_df['News'].tolist()
test_corpus = test_data['News'].tolist()

print(raw_docs_train[:10])

num_classes = 2
processed_docs_train = []

stop_words = set(stopwords.words('english'))

for doc in tqdm(raw_docs_train):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_train.append(" ".join(filtered))
processed_docs_test = []

for doc in tqdm(raw_docs_test):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_test.append(" ".join(filtered))

processed_test_sample = []
for doc in tqdm(test_corpus):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_test_sample.append(" ".join(filtered))



  0%|          | 83/31801 [00:00<00:38, 826.95it/s]

['Among 53 analysts covering Apple Inc. ( NASDAQ:AAPL ), 34 have Buy rating, 1 Sell and 18 Hold. Therefore 64% are positive. Apple Inc. had 381 analyst reports since July 21, 2015   BMO Capital Markets maintained Apple Inc. (NASDAQ:AAPL) on Monday, July 24 with “Buy” rating. The stock has “Hold” rating by Rosenblatt on Tuesday, August 22. The stock of Apple Inc. (NASDAQ:AAPL) has “Hold” rating given on Wednesday, October 28 by Mizuho. On Tuesday, February 13 the stock rating was upgraded by Bank of America to “Buy”. The firm earned “Market Perform” rating on Friday, February 16 by Raymond James. The stock of Apple Inc. (NASDAQ:AAPL) has “Hold” rating given on Tuesday, August 1 by Wells Fargo. The company was maintained on Wednesday, October 26 by Drexel Hamilton. The firm earned “Hold” rating on Monday, September 18 by Mizuho. The stock of Apple Inc. (NASDAQ:AAPL) has “Buy” rating given on Friday, July 7 by Cowen & Co. Jefferies maintained it with “Hold” rating and $130 target in Tuesd

100%|██████████| 711/711 [00:00<00:00, 937.25it/s]


In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(raw_docs_train)

86037


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  

vocab_length = len(tokenizer.word_index) + 1

print(vocab_length)

44596


In [ ]:
MAX_NB_WORDS = vocab_length
max_seq_len = 1000

word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_seq_testsample = tokenizer.texts_to_sequences(processed_test_sample)
word_index = tokenizer.word_index

word_seq_train = pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = pad_sequences(word_seq_test, maxlen=max_seq_len)
word_seq_testsample = pad_sequences(word_seq_testsample, maxlen=max_seq_len)

print(len(word_index))

44595


In [ ]:
import requests, zipfile, io
zip_file_url = "https://nlp.stanford.edu/data/glove.6B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print("GloVE available locally")

GloVE available locally


In [ ]:
import codecs

embeddings_index = {}
#f = codecs.open(‘crawl-300d-2M.vec’, encoding=’utf-8')
# for Glove
f = codecs.open('glove.6B.50d.txt', encoding='utf-8')
for line in tqdm(f):
  values = line.rstrip().rsplit(' ')
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

400000it [00:12, 31000.96it/s]


In [ ]:
words_not_found = []
embed_dim = 50
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
  if i >= nb_words:
     continue
  embedding_vector = embeddings_index.get(word)
  
  if (embedding_vector is not None) and len(embedding_vector) > 0:
     embedding_matrix[i] = embedding_vector
  else:
     words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 11955


In [ ]:
from keras.layers import BatchNormalization
from keras.layers import Embedding, Bidirectional, Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D, Activation
from keras.initializers import Constant

model = tf.keras.Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix],trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 50)          2229800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 50)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 996, 64)           16064     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 249, 64)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 2,311,965
Trainable params: 82,165
Non-trainable params: 2,229,800
_______________________________________

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

labels = to_categorical(np.asarray(train_df['Movement']))
y_train = np.array(train_df['Movement'])

print(labels[:10])
print(y_train[:10])


[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[0 1 0 1 0 1 0 1 0 1]


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es_callback = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(word_seq_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
795/795 [==============================] - 226s 282ms/step - loss: 0.6992 - accuracy: 0.4962 - val_loss: 0.6923 - val_accuracy: 0.5149
Epoch 2/10
795/795 [==============================] - 223s 281ms/step - loss: 0.6930 - accuracy: 0.5158 - val_loss: 0.6923 - val_accuracy: 0.5169
Epoch 3/10
795/795 [==============================] - 225s 283ms/step - loss: 0.6902 - accuracy: 0.5297 - val_loss: 0.6928 - val_accuracy: 0.5155
Epoch 4/10
795/795 [==============================] - 226s 284ms/step - loss: 0.6865 - accuracy: 0.5462 - val_loss: 0.6961 - val_accuracy: 0.5180
Epoch 5/10
795/795 [==============================] - 225s 284ms/step - loss: 0.6806 - accuracy: 0.5665 - val_loss: 0.7013 - val_accuracy: 0.5210
Epoch 6/10
795/795 [==============================] - 225s 283ms/step - loss: 0.6736 - accuracy: 0.5727 - val_loss: 0.6999 - val_accuracy: 0.5219
Epoch 7/10
795/795 [==============================] - 225s 283ms/step - loss: 0.6634 - accuracy: 0.5921 - val_loss: 0.7021 -

In [ ]:
y_test1 = np.array(test_df['Movement'])
y_test2 = to_categorical(np.asarray(test_df['Movement']))
loss, accuracy = model.evaluate(word_seq_test, y_test1, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 51.980883


In [50]:
test_data["Movement"].replace({-1: 0}, inplace=True)
y_testsample = np.array(test_data['Movement'])
loss, accuracy = model.evaluate(word_seq_testsample, y_testsample, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 67.369902
